In [ ]:
# Jupyter Notebook: test.ipynb
# 각 셀을 순서대로 실행하세요

# ============================================================
# Cell 1: Import Modules
# ============================================================
import os
import torch
import numpy as np
from pathlib import Path
import time
import json
from tqdm import tqdm
import importlib

# Utils import
from utils import create_dataloaders, CDMetrics, get_loss_fn


# ============================================================
# Cell 2: GPU 설정
# ============================================================
# 단일 GPU 사용
GPU_ID = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_ID)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using GPU: {GPU_ID}")
print(f"Device: {DEVICE}")


# ============================================================
# Cell 3: 실험 설정
# ============================================================
# 데이터셋 설정
DATASET_ROOT = "./dataset"
test_dataset = 'LEVIR-CD+'

# 모델 설정
test_model = 'A2Net'
use_base = True  # True: base version, False: full version

# 배치 크기
BATCH_SIZE = 16
NUM_WORKERS = 4
IMG_SIZE = 256

print(f"Dataset: {test_dataset}")
print(f"Model: {test_model}")
print(f"Base version: {use_base}")


# ============================================================
# Cell 4: 경로 설정
# ============================================================
# 실험 디렉토리
exp_dir = Path(f"experiments/{test_dataset}/{test_model}")
checkpoint_dir = exp_dir / "checkpoints"
best_model_path = checkpoint_dir / "best_model.pth"

print(f"Experiment directory: {exp_dir}")
print(f"Checkpoint path: {best_model_path}")

if not best_model_path.exists():
    raise FileNotFoundError(f"Best model not found at {best_model_path}")


# ============================================================
# Cell 5: 모델 로드
# ============================================================
def get_model_class(model_name, use_base=False):
    """모델 동적 import"""
    model_mapping = {
        'A2Net': {
            'base': ('models.a2net_base', 'A2NetBase'),
            'full': ('models.a2net', 'A2Net')
        },
        'Change3D': {
            'base': ('models.change3d_base', 'Change3DBase'),
            'full': ('models.change3d', 'Change3D')
        },
    }
    
    if model_name not in model_mapping:
        raise ValueError(f"Model {model_name} not implemented")
    
    version = 'base' if use_base else 'full'
    module_path, class_name = model_mapping[model_name][version]
    
    try:
        module = importlib.import_module(module_path)
        model_class = getattr(module, class_name)
        return model_class
    except ImportError:
        if use_base:
            raise ImportError(f"{model_name} base version not found")
        else:
            print(f"Full version not found, falling back to base")
            return get_model_class(model_name, use_base=True)

# 모델 생성
ModelClass = get_model_class(test_model, use_base=use_base)
model = ModelClass(num_classes=1).to(DEVICE)

# 체크포인트 로드
checkpoint = torch.load(best_model_path, map_location=DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"\n✓ Model loaded: {ModelClass.__name__}")
print(f"  Epoch: {checkpoint.get('epoch', 'unknown')}")
print(f"  Best F1: {checkpoint.get('best_f1', 0):.4f}")
print(f"  Best IoU: {checkpoint.get('best_iou', 0):.4f}")


# ============================================================
# Cell 6: 데이터로더 생성
# ============================================================
# 테스트 데이터로더만 생성
_, _, test_loader = create_dataloaders(
    root_dir=DATASET_ROOT,
    dataset_name=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    img_size=IMG_SIZE,
    augment=False
)

print(f"Test batches: {len(test_loader)}")
print(f"Total test samples: {len(test_loader.dataset)}")


# ============================================================
# Cell 7: 손실 함수
# ============================================================
criterion = get_loss_fn('bce_dice')
print("Loss function: BCEDiceLoss")


# ============================================================
# Cell 8: 테스트 평가
# ============================================================
print("\n" + "="*60)
print("Testing on full test set")
print("="*60)

test_metrics = CDMetrics()
test_loss = 0

# 추론 시간 측정용
inference_times = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        img1 = batch['img1'].to(DEVICE)
        img2 = batch['img2'].to(DEVICE)
        label = batch['label'].to(DEVICE)
        
        # Loss 계산
        output = model(img1, img2)
        loss = criterion(output, label)
        test_loss += loss.item()
        
        # 메트릭 업데이트
        test_metrics.update(output, label)
        
        # 추론 시간 측정 (개별 샘플)
        for i in range(img1.size(0)):
            torch.cuda.synchronize()
            
            start_time = time.time()
            _ = model(img1[i:i+1], img2[i:i+1])
            
            torch.cuda.synchronize()
            
            inference_times.append(time.time() - start_time)

avg_test_loss = test_loss / len(test_loader)
test_results = test_metrics.get_metrics()

print(f"\n{'Test Results':^60}")
print("="*60)
print(f"  Loss:      {avg_test_loss:.4f}")
print(f"  F1 Score:  {test_results['f1']:.4f}")
print(f"  IoU:       {test_results['iou']:.4f}")
print(f"  Precision: {test_results['precision']:.4f}")
print(f"  Recall:    {test_results['recall']:.4f}")
print(f"  OA:        {test_results['oa']:.4f}")
print(f"  Kappa:     {test_results['kappa']:.4f}")
print("="*60)


# ============================================================
# Cell 9: 추론 속도 분석
# ============================================================
print(f"\n{'Inference Speed Analysis':^60}")
print("="*60)

inference_times = np.array(inference_times)
avg_time = np.mean(inference_times)
std_time = np.std(inference_times)

# FPS 계산
fps = 1.0 / avg_time

# 이미지당 처리 시간 (ms)
inference_time_ms = avg_time * 1000

print(f"  Total samples:     {len(inference_times)}")
print(f"  Inference Time:    {inference_time_ms:.2f} ms/image")
print(f"  Std deviation:     {std_time*1000:.2f} ms")
print(f"  FPS:               {fps:.2f} images/sec")
print("="*60)


# ============================================================
# Cell 10: 배치 단위 추론 속도
# ============================================================
print(f"\n{'Batch Inference Speed':^60}")
print("="*60)

# Warmup
warmup_batch = next(iter(test_loader))
img1_warmup = warmup_batch['img1'].to(DEVICE)
img2_warmup = warmup_batch['img2'].to(DEVICE)

for _ in range(5):
    with torch.no_grad():
        _ = model(img1_warmup, img2_warmup)

torch.cuda.synchronize()

# 실제 측정 (10 배치)
num_measure_batches = min(10, len(test_loader))
batch_times = []

test_iter = iter(test_loader)
for _ in range(num_measure_batches):
    batch = next(test_iter)
    img1 = batch['img1'].to(DEVICE)
    img2 = batch['img2'].to(DEVICE)
    
    torch.cuda.synchronize()
    
    start = time.time()
    
    with torch.no_grad():
        _ = model(img1, img2)
    
    torch.cuda.synchronize()
    
    batch_times.append(time.time() - start)

avg_batch_time = np.mean(batch_times)
batch_fps = BATCH_SIZE / avg_batch_time
single_image_time = avg_batch_time / BATCH_SIZE

print(f"  Batch size:        {BATCH_SIZE}")
print(f"  Batches measured:  {num_measure_batches}")
print(f"  Avg batch time:    {avg_batch_time*1000:.2f} ms")
print(f"  Batch FPS:         {batch_fps:.2f} images/sec")
print(f"  Per image:         {single_image_time*1000:.2f} ms")
print("="*60)


# ============================================================
# Cell 11: 모델 정보
# ============================================================
print(f"\n{'Model Information':^60}")
print("="*60)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"  Total parameters:      {total_params:,}")
print(f"  Trainable parameters:  {trainable_params:,}")
print(f"  Model size:            {total_params * 4 / 1024 / 1024:.2f} MB (FP32)")
print("="*60)


# ============================================================
# Cell 12: 결과 저장
# ============================================================
results_dict = {
    'model': test_model,
    'dataset': test_dataset,
    'test_metrics': {
        'loss': float(avg_test_loss),
        'f1': float(test_results['f1']),
        'iou': float(test_results['iou']),
        'precision': float(test_results['precision']),
        'recall': float(test_results['recall']),
        'oa': float(test_results['oa']),
        'kappa': float(test_results['kappa'])
    },
    'inference_speed': {
        'inference_time_ms': float(inference_time_ms),
        'fps': float(fps),
        'std_time_ms': float(std_time * 1000),
        'batch_time_ms': float(avg_batch_time * 1000),
        'batch_fps': float(batch_fps)
    },
    'model_info': {
        'total_params': int(total_params),
        'trainable_params': int(trainable_params),
        'model_size_mb': float(total_params * 4 / 1024 / 1024)
    }
}

# JSON 저장
results_path = exp_dir / 'test_results.json'
with open(results_path, 'w') as f:
    json.dump(results_dict, f, indent=2)

print(f"\n✓ Results saved to {results_path}")

print("\n" + "="*60)
print("Test Complete!")
print("="*60)


# ============================================================
# Cell 13: 결과 요약
# ============================================================
print(f"\n{'SUMMARY':^60}")
print("="*60)
print(f"Model: {test_model}")
print(f"Dataset: {test_dataset}")
print(f"\nPerformance:")
print(f"  F1: {test_results['f1']:.4f}")
print(f"  IoU: {test_results['iou']:.4f}")
print(f"\nSpeed:")
print(f"  Inference Time: {inference_time_ms:.2f} ms")
print(f"  FPS: {fps:.2f}")
print(f"\nModel:")
print(f"  Parameters: {total_params:,}")
print(f"  Size: {total_params * 4 / 1024 / 1024:.2f} MB")
print("="*60)


Inference & Speed Measurement


NameError: name 'checkpoint_dir' is not defined